# Ejercicio I
Con este ejercicio vamos a revisitar el ejercicio 2 del viernes y lo expandimos por la visulaización de unos datos vectoriales (puntos). Usa este script para (1) abrir un mapa Google Earth Engine con los datos del Global Forest Watch, y (2) visualizar unos puntos, (3) extraer unos datos en las ubicaciónes de puntos.

## Parte I:
En la parte I primero visualizamos los datos de Global Forest Watch en una area de etudio (igual como lo hicimos el viernes). Después visualizamos algunos puntos. Tienes dos opciones:

1.  digitaliza algunos puntos en el mapa y guárdalos como una *ee.FeatureCollection*
2.  Carga un shapefile tuyo y conviertelo a una ee.FeatureCollection(), visualiza los puntos en el mapa

Cuando termines esta parte, puedes avanzar a parte 2.

### Carga las bibliotecas necesarias para este ejercicio, autentícate con GEE y activa las funciones de Matthias

In [1]:
import ee
import geemap
import eemont
from osgeo import ogr, osr

In [2]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [3]:
def fcTOshp(fc_geojson, outName):
    '''
    Function that takes the output from the getInfo of a ee.FeatureCollection() and
    converts it into a local shapefile. Code was generated through ChatGPT
    !!!! Works so far only for homogenous shapefiles (i.e., Point OR Polygon OR LineString!!!
    
    Arguments:
    ----------
    fc_geojson (required): output from getInfo() calling to a featureCollection
    outName (string, required): Name of the desired output-file
    
    '''
    # Check which geomtry type we have
    firstFeat = fc_geojson['features'][0]
    firstGeom = firstFeat['geometry']['type']   
    # Add the spatial reference
    sr = osr.SpatialReference()
    sr.ImportFromEPSG(4326)
    # Open the output shapefile using OGR
    driver = ogr.GetDriverByName('ESRI Shapefile')
    shapefile = driver.CreateDataSource(outName)
    if firstGeom == 'Polygon':
        layer = shapefile.CreateLayer(outName.split('.')[0], sr, geom_type=ogr.wkbPolygon)
    if firstGeom == 'Point':
        layer = shapefile.CreateLayer(outName.split('.')[0], sr, geom_type=ogr.wkbPoint)
    if firstGeom == 'LineString':
        layer = shapefile.CreateLayer(outName.split('.')[0], sr, geom_type=ogr.wkbLineString)
    # Get the feature fields and add them to the output shapefile attribute table
    for field in fc_geojson['features'][0]['properties'].keys():
        layer.CreateField(ogr.FieldDefn(field, ogr.OFTString))
    # Add the feature geometries and attributes to the output shapefile
    for feature in fc_geojson['features']:
        geom = ogr.CreateGeometryFromJson(str(feature['geometry']))
        feat = ogr.Feature(layer.GetLayerDefn())
        feat.SetGeometry(geom)
        for field in feature['properties'].keys():
            feat.SetField(field, feature['properties'][field])
        layer.CreateFeature(feat)
        feat = None
    # Close the output shapefile
    shapefile = None
    layer = None
    
def shpTOfc(pathToShape):
    import json
    # Open the shapefile, get the layer
    ds = ogr.Open(pathToShape)
    lyr = ds.GetLayer()
    # Create an emtpy feature collection
    fc = ee.FeatureCollection([])
    # Get the feature fields and geometries and add them to the feature collection
    idCount = 1
    for feat in lyr:
        geom = feat.geometry()
        # Automatic coordinate transformation
        from_sr = lyr.GetSpatialRef()
        to_sr = osr.SpatialReference()
        to_sr.ImportFromEPSG(4326)
        cs = osr.CoordinateTransformation(from_sr, to_sr)
        geom.Transform(cs)
        geojson = json.loads(geom.ExportToJson())
        properties = {}
        for i in range(feat.GetFieldCount()):
            field_name = feat.GetFieldDefnRef(i).GetName()
            field_value = feat.GetField(i)
            properties[field_name] = field_value
        ee_feat = ee.Feature(ee.Geometry(geojson), properties)
        # Remove the 'id' property from the feature
        ee_feat = ee_feat.set('id', idCount)
        idCount += 1
        fc = fc.merge(ee.FeatureCollection(ee_feat))
    return fc
def fcTOcsv(fc, outPath):
    import pandas as pd
    flag = 0
    featVals = fc['features']
    for f in featVals:
        prop = f['properties']
        if flag == 0:
            # Create a pandas data frame with the first element of the values
            out_pd = pd.DataFrame(prop, index=[0])
            flag = 1
        else:
            out_pd = pd.concat([out_pd, pd.DataFrame(prop, index=[0])])
    # Write the pandas dataframe 
    out_pd.to_csv(outPath, index=False)  

### Carga un mapa y encuentra tu area de estudio
* Ejecute la celda de código siguiente y navegue hasta la región de estudio de su interés
* Utilice las herramientas de digitalización para seleccionar un área de estudio de su elección
* A continuación, ejecute la siguiente celda de código para almacenar el área de estudio como ```ee.Geometry()```

In [4]:
Map = geemap.Map(zoom=4)
Map.add_basemap('SATELLITE') # add Google Earth Basemap
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [5]:
roi = Map.draw_last_feature.geometry()

In [6]:
# Global Forest Watch
gfw = ee.Image("UMD/hansen/global_forest_change_2021_v1_9").clip(roi)
Map.addLayer(gfw, {'bands': ['treecover2000'], 'min': 0, 'max': 100, 'palette': ['black', 'green']}, 'GFW Tree Cover')
Map.addLayer(gfw, {'bands': ['lossyear'], 'min': 0, 'max': 21, 'palette': ['yellow', 'red']}, 'GFW TC loss')

In [7]:
# Topography
elev = ee.Image("USGS/SRTMGL1_003").select(['elevation']).clip(roi)
slope = ee.Terrain.slope(elev)
Map.addLayer(elev, {'min': 200, 'max': 300, 'palette': ['black', 'white']}, 'Elevation')
Map.addLayer(slope, {'min': 0, 'max': 1, 'palette': ['black', 'white']}, 'Slope')

### Visualización de datos vectoriales en el mapa como una ee.FeatureCollection()
Usa opción I o opción II

### Opción I: digitalización manual
Borra el polígono del mapa usando el button del borrador. Después digitaliza unos puntos y corre el código abajo. Investiga la estructura de la variable *puntos*.

In [ ]:
# Convertir los puntos en una featureCollection
fc = ee.FeatureCollection(Map.draw_features)
fc

In [ ]:
# Bajarlo y guárdalo en un variable
puntos = fc.getInfo()
puntos

In [ ]:
# Si quieres, puedes bajar los puntos en tu disco duro
yourPath = ".shp"
fcTOshp(puntos, yourPath)

### Opción II: carga un shapefile tuyo (puntos)
carga un shapefile de puntos tuyo y aplica la función ```shpTOfc(pathToShape)``` para convertirlo en una ```ee.FeatureCollection```. Después visualiza el shapefile en el mapa

In [ ]:
path = ""
ownFC = shpTOfc(path)
ownFC

In [ ]:
Map.addLayer(ownFC, {'color': '000000', 'width': 1,'lineType': 'solid','fillColor': '00000000', 'propertyName':id}, 'My Points')

## Parte II: Extracción de datos en las ubicaciones de puntos
En esta parte aplicamos algunos funciones de *GEE* para extraer los valors de rasters en las ubicaciones de los puntos. Después guardamos los datos en tu disco duro. Primero usa el código de la pantalla para combinar las imagenes en una nueva imagen. Despues extrae los datos y usa la función ```fcTOcsv(json, outPath)``` para guardarlos.